✿ Orijinal veri setindeki sınıfları görüntülüyorum.

In [ ]:
import os

base_dir = "/kaggle/input/sea-animals-image-dataste"
print(os.listdir(base_dir))

✿ Her sınıfa ait ne kadar görsel olduğuna da baktım.

In [ ]:
for c in os.listdir(base_dir):
    c_path = os.path.join(base_dir, c)
    print(f"{c}: {len(os.listdir(c_path))} adet görsel")

✿ Aynı işlemleri iki farklı sınıf içeren kendi veri setim için de yaptım.

In [3]:
import os

base_dir = "/kaggle/input/instructor-sea-animals-photos-dataset"
print(os.listdir(base_dir))

['instructor-sea-animals-photos-dataset']


In [ ]:
for c in os.listdir(base_dir):
    c_path = os.path.join(base_dir, c)
    print(f"{c}: {len(os.listdir(c_path))} adet görsel")

# Veri Setlerini Birleştirme

✿  Orijinal veri setini working alanında oluşturduğum 'merged_dataset' klasörüne kopyalıyorum.

In [ ]:
import shutil
import os


src = "/kaggle/input/sea-animals-image-dataste"

dst = "/kaggle/working/merged_dataset"

os.makedirs(dst, exist_ok=True)
shutil.copytree(src, dst, dirs_exist_ok=True)

print("Orijinal dataset merged_dataset içine kopyalandı.")

✿ Kendi veri setimde bulunan `InstructorCorals` sınıfındaki tüm görselleri `merged_dataset` içerisindeki `Corals` sınıfına kopyalıyorum.

In [ ]:
import os
import shutil

src = "/kaggle/input/instructor-sea-animals-photos-dataset/InstructorCorals"
dst = "/kaggle/working/merged_dataset/Corals"

os.makedirs(dst, exist_ok=True)

for img in os.listdir(src):
    src_path = os.path.join(src, img)
    dst_path = os.path.join(dst, img)
    shutil.copy(src_path, dst_path)

print("InstructorCorals görselleri merged_dataset/Corals içine kopyalandı.")

✿ Aynı işlemi `InstructorFish` sınıfındaki görseller için de yapıyorum. Bu kez görseller `merged_dataset`'te bulunan `Fish` sınıfına kopyalanıyor.

In [ ]:
import os
import shutil

src_fish   = "/kaggle/input/instructor-sea-animals-photos-dataset/InstructorFish"
dst_fish   = "/kaggle/working/merged_dataset/Fish"

os.makedirs(dst_fish, exist_ok=True)

for img in os.listdir(src_fish):
    shutil.copy(os.path.join(src_fish, img), os.path.join(dst_fish, img))

print("InstructorFish görselleri merged_dataset/Fish içine kopyalandı.")

✿ working içinde bulunan `merged_dataset`'i kalıcı bir dataset haline getirebilmek için zip dosyası haline getirip kendi bilgisayarıma yükledikten sonra input alanına `my-merged-dataset` adı ile upload ediyorum. Böylece kendi veri setim ile orijinal veri setini birleştirdiğim bir veri seti oluşturmuş oluyorum.

In [ ]:
import shutil

folder_to_zip = "/kaggle/working/merged_dataset"

output_zip = "/kaggle/working/merged_dataset.zip"

shutil.make_archive("/kaggle/working/merged_dataset", 'zip', folder_to_zip)

# Görselleri Temizleme ve Standart Formata Dönüştürme

✿ Bu hücrede, `my-merged-dataset` klasöründeki tüm görseller:

- **RGB renk formatına** dönüştürülüyor,  
- **224x224 boyutuna** yeniden boyutlandırılıyor,  
- **JPEG formatında** ve yüksek kalite (quality=95) ile `cleaned_dataset` klasörüne kaydediliyor.
   
✿ Her sınıf için ayrı klasörler oluşturuluyor ve dosya isimleri korunuyor.  
✿ Hatalı veya açılamayan görselleri ekrana yazdırarak da işlem sırasında gözlemleyebiliyorum.  


In [ ]:
import os
from PIL import Image


input_dir = "/kaggle/input/my-merged-dataset"
output_dir = "/kaggle/working/cleaned_dataset"

os.makedirs(output_dir, exist_ok=True)

for class_name in os.listdir(input_dir):
    class_path = os.path.join(input_dir, class_name)
    if not os.path.isdir(class_path):
        continue

    output_class_path = os.path.join(output_dir, class_name)
    os.makedirs(output_class_path, exist_ok=True)

    for img_name in os.listdir(class_path):
        try:
            img_path = os.path.join(class_path, img_name)
            img = Image.open(img_path).convert("RGB")
            img = img.resize((224, 224))       
            new_name = os.path.splitext(img_name)[0] + ".jpg"
            save_path = os.path.join(output_class_path, new_name)
            img.save(save_path, "JPEG", quality=95)
        except Exception as e:
            print(f"Hata: {img_name}, {e}")

✿ Temizlenmiş veri setimi kaybetmemek için tekrar zip dosyası haline getirip kendi bilgisayarıma indirdikten sonra input dataset alanına `cleaned-dataset`adı ile upload ediyorum.

In [ ]:
import shutil

folder_to_zip = "/kaggle/working/cleaned_dataset"
output_zip = "/kaggle/working/cleaned_dataset.zip"

shutil.make_archive("/kaggle/working/cleaned_dataset", 'zip', folder_to_zip)

# Dataset'i Train/Validation/Test Olarak Bölme

✿ Bu hücrede`cleaned_dataset` klasöründeki verileri **train**, **val** ve **test** setleri olarak ayırıyorum. Ardında ayrılmış dataseti working alanına dataset_split olarak kaydediyorum.

   `train_ratio = 0.7` -> Verilerin %70'i eğitim için.  
   `val_ratio = 0.15` -> Verilerin %15'i doğrulama için.  
   `test_ratio = 0.15` -> Verilerin %15'i test için.  

✿ Her sınıf için resimler rastgele karıştırılıyor ve ilgili klasörlere kopyalanıyor.  
  
✿ Sonuç olarak model eğitiminde ve değerlendirmesinde kullanılacak verileri düzenli bir şekilde ayırmış oluyorum.

In [ ]:
import os
import shutil
import random

source_dir = "/kaggle/working/cleaned_dataset"

split_dir = "/kaggle/working/dataset_split"
os.makedirs(split_dir, exist_ok=True)

train_ratio = 0.7
val_ratio = 0.15
test_ratio = 0.15

for class_name in os.listdir(source_dir):
    class_path = os.path.join(source_dir, class_name)
    if not os.path.isdir(class_path):
        continue

    imgs = [f for f in os.listdir(class_path) if f.lower().endswith(".jpg")]
    random.shuffle(imgs)

    n_total = len(imgs)
    n_train = int(n_total * train_ratio)
    n_val = int(n_total * val_ratio)
    n_test = n_total - n_train - n_val

    splits = {
        "train": imgs[:n_train],
        "val": imgs[n_train:n_train+n_val],
        "test": imgs[n_train+n_val:]
    }

    for split_name, split_imgs in splits.items():
        split_class_dir = os.path.join(split_dir, split_name, class_name)
        os.makedirs(split_class_dir, exist_ok=True)
        for img_name in split_imgs:
            src_path = os.path.join(class_path, img_name)
            dst_path = os.path.join(split_class_dir, img_name)
            shutil.copy(src_path, dst_path)

print("Dataset train/val/test olarak ayrıldı ve dataset_split klasörüne kaydedildi.")

✿ Kalıcılık adına bu klasörü de zip dosyası haline getirip kendi bilgisayarıma indirdikten sonra input dataset alanına `split-dataset` adı ile upload ediyorum.

In [ ]:
import shutil

folder_to_zip = "/kaggle/working/dataset_split"
output_zip = "/kaggle/working/dataset_split.zip"

shutil.make_archive("/kaggle/working/dataset_split", 'zip', folder_to_zip)

# Train Seti Sınıf Dağılımının Görselleştirilmesi

✿ Data augmentation öncesi `dataset_split/train` klasöründeki her sınıfın görsel sayısını hesaplıyor ve grafik ile görselleştiriyorum.  
  
- `plt.xticks(rotation=90)` komutuyla sınıf isimlerini dikey göstererek okunabilirliği arttırıyorum.


In [ ]:
import os
import matplotlib.pyplot as plt

split_dir = "/kaggle/working/dataset_split/train"

class_counts = {}
for class_name in os.listdir(split_dir):
    class_path = os.path.join(split_dir, class_name)
    count = len([f for f in os.listdir(class_path) if f.lower().endswith(".jpg")])
    class_counts[class_name] = count

plt.figure(figsize=(12,6))
plt.bar(class_counts.keys(), class_counts.values(), color='teal')
plt.xticks(rotation=90)
plt.ylabel("Görsel Sayısı")
plt.title("Train Seti Sınıf Dağılımı")
plt.show()

# ImageDataGenerator ile Veri Ön İşleme ve Augmentation

✿ Bu hücrede `ImageDataGenerator` kullanarak train, validation ve test verilerini hazırlıyorum.  

* **Train seti** için uyguladığım **data augmentation** teknikleri:

    Normalizasyon (`rescale=1./255`)
    Döndürme (0–30°)  
    Yatay/dikey kaydırma (%10)  
    Zoom (%20)  
    Yatay çevirme  
    Parlaklık jitter  
    `fill_mode="nearest"` ile boşluk doldurma

✿ **Validation** ve **Test** setlerinde yalnızca **normalize** işlemi yapıyorum.  

✿ Sonrasında `flow_from_directory` metodu ile train, validation ve test klasörlerinden veri okuyarak `generator` objeleri oluşturuyorum. Bu sayede model eğitiminde mini-batch halinde veriler otomatik besleniyor.

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1./255,          
    rotation_range=30,       
    width_shift_range=0.1,   
    height_shift_range=0.1,  
    zoom_range=0.2,         
    horizontal_flip=True,    
    vertical_flip=False,     
    brightness_range=[0.8,1.2], 
    fill_mode="nearest"
)

# Validation ve test için sadece normalize 
val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_dir = "/kaggle/working/dataset_split/train"
val_dir   = "/kaggle/working/dataset_split/val"
test_dir  = "/kaggle/working/dataset_split/test"

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224,224),
    batch_size=32,
    class_mode="categorical"
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(224,224),
    batch_size=32,
    class_mode="categorical"
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224,224),
    batch_size=64,
    class_mode="categorical",
    shuffle=False
)

# Data Augmentation Sonuçlarını Görselleştirme

✿ Bu hücrede, `ImageDataGenerator` ile tanımlanan **data augmentation** işlemlerinin görsellere nasıl yansıdığı inceleyebilmek için görselleştirme yapıyorum.  

- Eğitim setinden bir batch alıp (32 görsel), ilk 9 görsel 3x3 grid halinde çizdiriliyor.  

✿ Bu sayede döndürme, kaydırma, zoom, yatay çevirme ve parlaklık değişikliklerinin örnek görseller üzerinde nasıl çalıştığını gözlemleyebiliyorum.  
✿ Modelin daha genelleştirilebilir hale gelmesi için augmentation etkileri burada görsel olarak kontrol edilmiş oluyor.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,      
    width_shift_range=0.1,   
    height_shift_range=0.1, 
    zoom_range=0.2,         
    horizontal_flip=True,   
    brightness_range=[0.8,1.2] 
)

train_generator = train_datagen.flow_from_directory(
    '/kaggle/working/dataset_split/train',
    target_size=(224,224),
    batch_size=32,
    class_mode='categorical'
)

x_batch, y_batch = next(train_generator)

plt.figure(figsize=(12, 12))
for i in range(9):
    plt.subplot(3, 3, i+1)
    plt.imshow(x_batch[i])
    plt.axis("off")
plt.show()

# Validation ve Test Veri Generator'larının Hazırlanması

✿ Bu hücrede, **validation** ve **test** setleri için `ImageDataGenerator` tanımlanıyor.  

* Sadece **rescale=1./255** uygulanıyor (normalize işlemi).  
* **Augmentation uygulamıyorum**, çünkü modelin doğrulama ve test performansı gerçekçi değerlendirilmeli.  
* `flow_from_directory` ile `val` ve `test` klasörlerinden veriler yükleniyor.  
* `shuffle=False` kullanılarak veri sırası korunuyor.  

✿ Kısaca model eğitiminden bağımsız olarak doğrulama ve test aşamalarında kullanılacak temiz veri akışlarını hazırlamış oluyorum.

In [ ]:
# Validation için sadece rescale
val_datagen = ImageDataGenerator(rescale=1./255)
val_generator = val_datagen.flow_from_directory(
    '/kaggle/working/dataset_split/val',
    target_size=(224,224),
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)

# Test için sadece rescale
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
    '/kaggle/working/dataset_split/test',
    target_size=(224,224),
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)

# Basit CNN Modeli Kurulumu ve Derleme

✿ Bu hücrede de Keras kullanrak sıfırdan bir **Convolutional Neural Network (CNN)** modeli oluşturuyorum.

* **Girdi**: (224x224x3) boyutunda renkli görseller  
* **Katmanlar**:  
  -> 3 adet Conv2D + MaxPooling blokları  
  -> Flatten -> Dense katmanı bağlantısı  
  -> 256 nöronlu Dense + Dropout(0.5)  
  ->`softmax` çıkış katmanı (23 sınıf için)  

* **Derleme (compile)**:  
  -> Optimizer: **Adam** (learning_rate=0.0001)  
  -> Loss: **categorical_crossentropy**  
  ->Metric: **accuracy**  


✿ Son adımda `model.summary()` çıktısı, her katmanın ismini, çıktı boyutunu ve öğrenilebilir parametre sayısını tablo halinde gösteriyor.   

In [ ]:
from tensorflow.keras import layers, models, optimizers

num_classes = 23 

model = models.Sequential([
    # 1. Convolution + Pooling
    layers.Conv2D(32, (3,3), activation='relu', input_shape=(224,224,3)),
    layers.MaxPooling2D((2,2)),
    
    # 2. Convolution + Pooling
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),
    
    # 3. Convolution + Pooling
    layers.Conv2D(128, (3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),
    
    layers.Flatten(),         
    
    layers.Dense(256, activation='relu'), 
    layers.Dropout(0.5),                  
    
    layers.Dense(num_classes, activation='softmax') 
])

model.compile(
    optimizer=optimizers.Adam(learning_rate=0.0001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

# Modelin Eğitilmesi ve Eğitim Sürecinin Görselleştirilmesi

✿ Bu hücrede CNN modelinin, eğitim ve doğrulama setleri üzerinden **25 epoch** boyunca eğitilmesini sağladım.  
`history` objesi, eğitim sırasında kaydedilen accuracy ve loss değerlerini saklıyor.  

- **Sol grafik**: Eğitim ve doğrulama accuracy (başarı oranı)  
- **Sağ grafik**: Eğitim ve doğrulama loss (kayıp)  

Bu görseller sayesinde:  
- Modelin **öğrenme eğrisi** takip edilebiliyor.  
- **Overfitting/underfitting** belirtileri gözlemlenebiliyor (örn. doğrulama kaybının artması).

In [ ]:
import matplotlib.pyplot as plt

epochs = 25
batch_size = 64 

history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_data=val_generator,
    validation_steps=val_generator.samples // batch_size,
    epochs=epochs
)

plt.figure(figsize=(12,5))

# Accuracy
plt.subplot(1,2,1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Val Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

# Loss
plt.subplot(1,2,2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.show()

# Pre-trained VGG16 ile Transfer Learning Modeli

Henüz yeterli veriye ulaşamadığımı düşündüğüm için :') **pre trained VGG16** kullanarak transfer learning uyguluyorum. Böylece daha az veriyle yüksek performans elde edilebilir.

1. **Base Model**  
   - `VGG16(weights='imagenet', include_top=False)` ile ImageNet üzerinde eğitilmiş ağırlıklar yükleniyor.  
   - `include_top=False` Orijinal sınıflandırma katmanları kaldırılıyor, sadece özellik çıkarıcı katmanlar alınıyor.  
   - `input_shape=(224,224,3)` Modelin giriş boyutu ayarlanıyor.  

2. **Base model dondurma**  
   - `base_model.trainable = False` ile önceden öğrenilmiş katmanlar eğitim sırasında güncellenmeyecek, sadece üst katmanlar eğitilecek.  

3. **Yeni üst katmanlar ekleme**  
   - Flatten -> Dense(256, relu) -> Dropout(0.5) -> Dense(23, softmax)  
   - Bu katmanlar datasetimizdeki sınıfları öğrenmek için eklendi.  

4. **Compile**  
   - Optimizer: Adam (lr=0.0001)  
   - Loss: Categorical Crossentropy  
   - Metric: Accuracy  

* `model.summary()` ile modelin mimarisi ve toplam parametre sayısı görüntüleniyor.  


In [ ]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras import layers, models, optimizers

# dataset için
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# görselleştirme için
import matplotlib.pyplot as plt
import os

In [ ]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224,224,3))

base_model.trainable = False

# Yeni üst katmanlar
model = models.Sequential([
    base_model,
    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(23, activation='softmax')
])

model.compile(
    optimizer=optimizers.Adam(learning_rate=0.0001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

# Transfer Learning Modelinin Eğitilmesi

✿ Bu hücrede, VGG16 tabanlı transfer learning modeli eğitim ve doğrulama setleri üzerinde **25 epoch** boyunca eğitiliyor.  

- `train_generator` -> Eğitim verilerini besler (augmentation + normalize)  
- `val_generator` -> Doğrulama verilerini besler (sadece normalize)  
- `steps_per_epoch` ve `validation_steps` -> Her epoch için kaç batch işleneceğini belirler  

✿ Eğitim sırasında `history` objesi, her epoch sonrası **loss** ve **accuracy** değerlerini saklar. Bu değerleri eğitim eğrilerini görselleştirmek için kullanacağım.

In [ ]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // 64,
    validation_data=val_generator,
    validation_steps=val_generator.samples // 64,
    epochs=25
)

# Transfer Learning Modeli Eğitim Sonuçlarının Görselleştirilmesi

✿ `Sol grafikte`, eğitim ve **accuracy** değerleri, `sağ grafikte` eğitim ve **loss** değerleri epoch bazında gösteriliyor.

In [ ]:
import matplotlib.pyplot as plt

# Accuracy grafiği
plt.figure(figsize=(12,5))
plt.subplot(1,2,1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

# Loss grafiği
plt.subplot(1,2,2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.show()

≽^•⩊•^≼ Bu görüntü hoşuma gittiği için modeli kendi bilgisayarıma yüklüyorum. ≽^•⩊•^≼

In [ ]:
model.save('/kaggle/working/sea_animals_cnn_vgg16.h5')

In [ ]:
import shutil

model_path = "/kaggle/working/sea_animals_cnn_vgg16.h5"
shutil.make_archive("/kaggle/working/sea_animals_cnn_vgg16.h5", 'zip', root_dir="/kaggle/working", base_dir="sea_animals_cnn_vgg16.h5")

# Test Seti Üzerinde Model Değerlendirmesi

✿ Bu hücrede, eğitilmiş model **test seti** üzerinde değerlendirilerek performansı detaylı olarak inceleyebiliriz.

1. **Tahminler (Predictions)**  
   - `model.predict(test_generator)` ile test setindeki tüm görseller için sınıf olasılıkları elde ediliyor.  
   - `np.argmax` ile en yüksek olasılığa sahip sınıf seçiliyor.  

2. **Confusion Matrix (Karışıklık Matrisi)**  
   - `confusion_matrix(y_true, y_pred)` ile gerçek ve tahmin edilen sınıflar karşılaştırılıyor.  
   - `seaborn` ile görselleştiriliyor.  
   - Matristeki yüksek değerler doğru sınıflandırmaları, yanlış sınıflandırmalar ise düşük değerlerle gösteriliyor.  

3. **Classification Report**  
   - `precision`, `recall`, `f1-score` ve `support` değerleri her sınıf için hesaplanıyor.  
   - Modelin hangi sınıflarda güçlü veya zayıf olduğu anlaşılabiliyor. 

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np
import seaborn as sns

# Test generatordan tüm veriyi tahmin et
y_pred_probs = model.predict(test_generator)
y_pred = np.argmax(y_pred_probs, axis=1)
y_true = test_generator.classes

# Confusion matrix
cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(15,12))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=test_generator.class_indices.keys(), yticklabels=test_generator.class_indices.keys())
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.title('Confusion Matrix')
plt.show()

# Classification report
print(classification_report(y_true, y_pred, target_names=test_generator.class_indices.keys()))

# Grad-CAM için Gerekli Kütüphanelerin İçe Aktarılması

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import cv2

# Eğitilmiş Modelin Yüklenmesi ve Katmanlarının İncelenmesi

✿ Daha önce eğittiğim CNN modeli (`sea_animals_cnn_vgg16.h5`) **yükleniyor**:

- `tf.keras.models.load_model(model_path)` ile model dosyası belleğe alınıyor.  
- `model.summary()` ile modelin tüm katmanları, çıktı boyutları ve parametre sayıları görüntüleniyor.  
- Katman isimleri ve yapısı, **Grad-CAM uygularken hangi convolutional katmanı hedefleyeceğimizi belirlemek** için önemli.  

In [ ]:
model_path = "/kaggle/working/sea_animals_cnn_vgg16.h5"
model = tf.keras.models.load_model(model_path)
model.summary()  # Hangi layer isimlerini kullanabileceğini görmek için

# Grad-CAM için Base Model ve Son Convolutional Katmanın Seçilmesi

✿ Bu hücrede Grad-CAM hesaplaması için gerekli model kısmı ve hedef katman tanımlıyorum.

1. **Base Model**  
   - `base_model = model.get_layer('vgg16')` -> Transfer learning kısmı olan VGG16 modeli alınıyor.  
   - `summary()` çıktısına göre katman ismi doğrulanıyor.  

2. **Son Convolutional Katman**  
   - `last_conv_layer_name = "block5_conv3"` -> Grad-CAM için son conv katman seçilir.  
   - Bu katman, modelin sınıflandırma kararına en çok katkıda bulunan feature map’leri içeriyor. 

In [ ]:
# Base model (VGG16 kısmı)
base_model = model.get_layer('vgg16')  
last_conv_layer_name = "block5_conv3" 

# Base Model’in Tensorlarının Hazırlanması için Dummy Çağrı

✿ Burada Grad-CAM hesaplaması için **modelin input ve output tensorlarını** hazırlıyorum.

- `dummy_input` ile (1, 224, 224, 3) boyutunda boş bir örnek tensor oluşturuluyor.  
- `base_model(dummy_input)` çağrısı ile modelin katmanları çalıştırılıyor ve tensor yapıları aktive ediliyor.  

In [ ]:
dummy_input = np.zeros((1, 224, 224, 3), dtype=np.float32)
_ = base_model(dummy_input)

# Grad-CAM Heatmap Hesaplama Fonksiyonu

✿ Modelin görüntü üzerinde **hangi bölgelere dikkat ettiğini** görselleştirmek için bir fonksiyon tanımlıyorum.

1. **Hedef Katman Çıkışı**  
     
2. **Gradyan Hesaplama**  
 
3. **Heatmap Oluşturma**  
     

In [ ]:
def get_gradcam_heatmap(img_array, base_model, last_conv_layer_name='block5_conv3'):
    last_conv_layer = base_model.get_layer(last_conv_layer_name)
    grad_model = tf.keras.models.Model(
        inputs=base_model.input,
        outputs=last_conv_layer.output
    )

    with tf.GradientTape() as tape:
        conv_outputs = grad_model(img_array)
        # Dummy prediction için feature maplerin toplamını kullanıyoruz
        class_channel = tf.reduce_mean(conv_outputs, axis=-1)

    grads = tape.gradient(class_channel, conv_outputs)
    pooled_grads = tf.reduce_mean(grads, axis=(0,1,2))
    conv_outputs = conv_outputs[0]
    heatmap = conv_outputs @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)
    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
    return heatmap.numpy()

# Grad-CAM Heatmap’in Görselleştirilmesi ve Kaydedilmesi

✿ Bu hücrede, hesaplanan Grad-CAM heatmap’in orijinal görselle birleştirilerek görselleştirilmesi sağlıyorum.

1. **Görselin Yüklenmesi**  
   - `cv2.imread` ile görüntü okunur ve RGB formatına çevriliyor.  

2. **Heatmap İşleme**  
   - Heatmap, orijinal görsel boyutuna yeniden boyutlandırılır.  
   - 0–255 aralığına normalize edilip renkli bir colormap (`COLORMAP_JET`) uygulanır.  

3. **Görsellerin Birleştirilmesi**  
   - `superimposed_img = heatmap * alpha + img` ile heatmap, orijinal görsele saydamlık (`alpha`) ile bindirilir.  

4. **Görselleştirme**  
   - `matplotlib` ile görsel çizdirilir ve eksenler gizlenir.  

In [ ]:
def save_and_display_gradcam(img_path, heatmap, alpha=0.4):
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    heatmap = cv2.resize(heatmap, (img.shape[1], img.shape[0]))
    heatmap = np.uint8(255 * heatmap)
    heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)

    superimposed_img = heatmap * alpha + img
    plt.figure(figsize=(6,6))
    plt.imshow(np.uint8(superimposed_img))
    plt.axis('off')
    plt.show()

✿ Grad-CAM için kullanılacak test görselini hazırlıyorum.

1. **Görselin Yüklenmesi**  
   - `load_img(test_img_path, target_size=(224,224))` ile görsel belirtilen boyuta yeniden boyutlandırılarak yüklenir.  

2. **Array’e Dönüştürme**  
   - `img_to_array(img)` ile NumPy array formatına çevrilir.  
   - `np.expand_dims(img_array, axis=0)` ile batch boyutu eklenir, modelin beklediği input shape sağlanır.  

3. **Normalize Etme**  
   - Piksel değerleri 0–1 aralığına getirilir ( veri ölçeklemesi uygunluğu için)   

In [ ]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array

test_img_path = "/kaggle/working/merged_dataset/Corals/coral1.jpg"
img = load_img(test_img_path, target_size=(224,224))
img_array = img_to_array(img)
img_array = np.expand_dims(img_array, axis=0) / 255.0

✿ Hazırladığım test görseli için Grad-CAM heatmap hesaplanıyor ve orijinal görsel üzerine bindirilerek gösteriliyor.

In [ ]:
heatmap = get_gradcam_heatmap(img_array, base_model, last_conv_layer_name)
save_and_display_gradcam(test_img_path, heatmap)

# Keras Tuner Kütüphanesinin Kurulması ve İçe Aktarılması

In [ ]:
!pip install keras-tuner -q
import keras_tuner as kt

# Keras Tuner ve Veri Hazırlığı için Kütüphanelerin İçe Aktarılması ve Parametrelerin Tanımlanması

Bu hücrede:

1. **Kütüphaneler yükleniyor**  
   - `tensorflow` ve `keras.preprocessing.image` → Model ve veri işlemleri için  
   - `keras_tuner` → Hiperparametre optimizasyonu için  
   - `numpy` ve `matplotlib.pyplot` → Veri işleme ve görselleştirme için  

2. **Dataset yolları belirleniyor**  
   - `train_dir` → Eğitim seti klasörü  
   - `val_dir` → Doğrulama seti klasörü  

3. **Hiperparametreler ve batch ayarları**  
   - `IMG_SIZE = (224, 224)` → Görsellerin yeniden boyutlandırılacağı boyut  
   - `BATCH_SIZE = 32` → Her batch’te işlenecek görsel sayısı (ε(´｡•᎑•`)っ☆ 64 seçmedim çünkü 32 de hafıza daha az kullanılıyor ve gradyan güncellemeleri daha sık)

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import keras_tuner as kt
import numpy as np
import matplotlib.pyplot as plt

train_dir = "/kaggle/working/dataset_split/train"
val_dir = "/kaggle/working/dataset_split/val"

IMG_SIZE = (224, 224)
BATCH_SIZE = 32

# Eğitim ve Doğrulama Verisi için Data Augmentation ve Normalizasyon

✿ Model eğitimini başlatmadan önce veriyi uygun formatta ve çeşitlilikte hazırlamak için, Keras Tuner ile kullanılacak eğitim ve doğrulama veri generatorlarını hazırlıyorum.

1. **Eğitim verisi için augmentation (`train_datagen`)**  
   - `rescale=1./255` -> Piksel değerlerini 0–1 aralığına getirir.  
   - `rotation_range`, `width_shift_range`, `height_shift_range`, `shear_range`, `zoom_range`, `horizontal_flip`-> Veri çeşitliliğini artırarak modelin daha genelleyici öğrenmesini sağlar.  
   - `fill_mode='nearest'` -> Görsel taşma/padding durumunda yeni piksellerin doldurma yöntemi.

2. **Doğrulama verisi için sadece normalizasyon (`val_datagen`)**  
   - Validation verisi augmentation uygulanmadan sadece normalize ediliyor, böylece gerçek doğrulama performansı ölçülür.

3. **Generator’lar (`flow_from_directory`)**  
   - `train_generator` ve `val_generator` -> Eğitim ve doğrulama verilerini batch batch modele besler.  
   - `target_size=IMG_SIZE` -> Görseller 224x224 boyutuna yeniden boyutlandırılır.  
   - `batch_size=BATCH_SIZE` -> Her batch’te işlenecek örnek sayısı  
   - `class_mode='categorical'` -> Çok sınıflı sınıflandırma için.



In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1./255)

# Generatorlar
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

# Keras Tuner için Model Oluşturma Fonksiyonu (`build_model`)

✿ Bu hücrede, **hiperparametre optimizasyonu için model oluşturma fonksiyonu** tanımlıyorum.

1. **Base Model**  
   - `VGG16` önceden eğitilmiş modeli yüklenir  
   - `base_model.trainable = False` -> Transfer learning uygulanır, base katmanlar eğitilmez.

2. **Yeni Üst Katmanlar**    
   - Bu katmanlar datasetimizdeki sınıfları öğrenmek için eklenir.

3. **Hiperparametreler (`hp`)**  
   - `learning_rate` → [0.001, 0.0001, 0.01] seçeneklerinden seçim yapılabilir.  
   - `optimizer_choice` → ['adam','rmsprop','sgd'] seçeneklerinden biri kullanılır.  
   - Seçilen optimizer ve learning rate ile model compile edilir.  

In [ ]:
def build_model(hp):
    base_model = tf.keras.applications.VGG16(
        input_shape=(224,224,3),
        include_top=False,
        weights='imagenet'
    )
    base_model.trainable = False  # Transfer learning

    model = tf.keras.Sequential([
        base_model,
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(train_generator.num_classes, activation='softmax')
    ])

    # Hiperparametreler
    learning_rate = hp.Choice('learning_rate', values=[1e-3, 1e-4, 1e-2])
    optimizer_choice = hp.Choice('optimizer', values=['adam','rmsprop','sgd'])

    if optimizer_choice == 'adam':
        optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    elif optimizer_choice == 'rmsprop':
        optimizer = tf.keras.optimizers.RMSprop(learning_rate=learning_rate)
    else:
        optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate)

    model.compile(optimizer=optimizer,
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

# Keras Tuner ile Random Search Tuner Oluşturma

✿ `RandomSearch` kullanarak **hiperparametre araması için tuner** oluşturuyorum. Bu adımı, farklı learning rate ve optimizer kombinasyonlarını otomatik olarak denemek ve en iyi performansı veren modeli bulmak için yapıyorum.

- `build_model` -> Hangi modelin ve hiperparametre aralığının kullanılacağını belirler.  
- `objective='val_accuracy'` -> Tuner, doğrulama doğruluk (validation accuracy) değerini maksimize etmeye çalışır.
- `max_trials=5` -> Tuner 5 farklı hiperparametre kombinasyonunu dener (deneme sayısı).  
- `directory='tuner_dir'` ve `project_name='all_classes_tuner'` ->  Arama sonuçları kaydediliyor ve proje organizasyonu sağlanıyor.  

In [ ]:
tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,
    directory='tuner_dir',
    project_name='all_classes_tuner'
)

# Keras Tuner ile Hiperparametre Aramasının Başlatılması

✿ Bu hücrede, `tuner.search` ile **model hiperparametreleri otomatik olarak deneniyor**

✿ Keras Tuner'in otomatik olarak optimizer ve learning rate gibi parametreleri test edip en iyi performansı veren modeli seçmesini sağlar.

In [ ]:
tuner.search(
    train_generator,
    validation_data=val_generator,
    epochs=5,
    verbose=1
)

✿ Doğrulama setimde yakaladığım başarı yetersiz olunca :') base modelin son birkaç katmanını açıp (fine-tuning) epoch sayımı artırıyorum.

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping
import keras_tuner as kt

# Veri Yükleme (DataLoader)

✿ Bu hücrede **`cleaned_dataset`** kullanarak eğitim ve doğrulama verileri hazırlıyorum.

- **`ImageDataGenerator`** ile veri artırma (augmentation) uyguluyorum. Böylece modelim hem veri çeşitliliği kazanacak hem de overfitting riskini azaltacak.

In [ ]:
img_size = (224, 224)
batch_size = 32

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)

train_generator = train_datagen.flow_from_directory(
    "cleaned_dataset",
    target_size=img_size,
    batch_size=batch_size,
    class_mode="categorical",
    subset="training"
)

val_generator = train_datagen.flow_from_directory(
    "cleaned_dataset",
    target_size=img_size,
    batch_size=batch_size,
    class_mode="categorical",
    subset="validation"
)

# HyperModel (Fine-Tuning Destekli)

✿ Keras Tuner kullanılarak **en iyi hiperparametre kombinasyonunu** bulmak için kullandığım bu hücrede  **VGG16 tabanlı transfer learning modeli** ve **hyperparameter tuning** için model mimarisi tanımlıyorum.

- **Fine-Tuning**
  - Son 4 katman eğitilebilir (`trainable=True`), diğer katmanlar donduruluyor (`trainable=False`).
  - Bu sayede model, önceden öğrenilmiş özellikleri korurken yeni veri setimize uyum sağlıyor.

In [ ]:
def build_model(hp):
    base_model = keras.applications.VGG16(
        include_top=False, 
        input_shape=(224,224,3),
        weights="imagenet"
    )

    # Son birkaç katmanı açalım (fine-tuning)
    for layer in base_model.layers[:-4]:
        layer.trainable = False
    for layer in base_model.layers[-4:]:
        layer.trainable = True

    x = layers.Flatten()(base_model.output)
    x = layers.Dense(
        units=hp.Choice("dense_units", [128, 256, 512]),
        activation="relu"
    )(x)
    x = layers.Dropout(hp.Float("dropout", 0.3, 0.6, step=0.1))(x)
    output = layers.Dense(train_generator.num_classes, activation="softmax")(x)

    model = keras.Model(inputs=base_model.input, outputs=output)

    model.compile(
        optimizer=hp.Choice("optimizer", ["adam", "rmsprop", "sgd"]),
        loss="categorical_crossentropy",
        metrics=["accuracy"]
    )
    return model

✿ *Keras Tuner** ile hiperparametre optimizasyonu için **Random Search** yöntemi tanımlıyorum.

✿ Bu tuner, **en iyi Dense layer boyutu, dropout oranı ve optimizer** kombinasyonunu belirlemek için kullanılıyor ve tüm sınıflar üzerinden çalışacak şekildedir.

- **Tuner Türü:**
  - `kt.RandomSearch` → rastgele hiperparametre kombinasyonlarını dener.
  
- **Parametreler:**
  - `build_model` → önceki hücrede tanımlanan HyperModel fonksiyonu.
  - `objective="val_accuracy"` → doğrulama doğruluğu en iyi olan modeli seçmek için hedef.
  - `max_trials=5` → toplam deneme sayısı.
  - `executions_per_trial=1` → her denemenin tek bir kez çalıştırılması.

In [ ]:
tuner = kt.RandomSearch(
    build_model,
    objective="val_accuracy",
    max_trials=5,
    executions_per_trial=1,
    directory="project_tuner_all_classes",
    project_name="class_tuning"
)

**Random Search Tuner** kullanılarak modelin en iyi hiperparametreleri belirleniyor ve eğitim başlatılıyor.

(づ๑•ᴗ•๑)づ ☆ Overfitting riskini azaltmak için EarlyStopping kullanıyorum.

- **EarlyStopping Callback:**
  - `monitor="val_loss"` → doğrulama kaybı izlenir.
  - `patience=3` → kayıp 3 epoch boyunca iyileşmezse eğitim durur.
  - `restore_best_weights=True` → durdurulduğunda en iyi ağırlıklar geri yüklenir.

In [ ]:
stop_early = EarlyStopping(monitor="val_loss", patience=3, restore_best_weights=True)

tuner.search(
    train_generator,
    validation_data=val_generator,
    epochs=8,   # daha uzun eğitim
    callbacks=[stop_early],
    verbose=1
)

# En İyi Hiperparametrelerin Alınması

In [ ]:
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
print(best_hps.values)

In [ ]:
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

print("En iyi Dense layer boyutu:", best_hps.get("dense_units"))
print("En iyi Dropout oranı:", best_hps.get("dropout"))
print("En iyi optimizer:", best_hps.get("optimizer"))

In [ ]:
model = tuner.hypermodel.build(best_hps)

# Modelin Eğitilmesi

✿ Bu hücrede, belirlenen hiperparametrelerle model **eğitim seti üzerinde eğitiyorum** ve doğrulama seti ile performansını izliyorum.

1. `train_generator` -> Modeli eğitmek için kullanılan eğitim verisi  
2. `validation_data=val_generator` -> Eğitim sırasında modelin doğrulama performansını ölçmek için kullanılır  
3. `epochs=25` -> Modelin tüm eğitim verisini 25 kez görmesi sağlanır  

> Bu adım, modelin gerçek veri üzerinde öğrenmesini ve doğrulama seti ile overfitting veya underfitting durumlarının kontrolünü sağlıyor. ☆

In [ ]:
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=25
)

# Eğitim ve Doğrulama Doğruluğu Grafiği

- `Mavi çizgi:` Eğitim doğruluğu (train accuracy)
- `Turuncu çizgi:` Doğrulama doğruluğu (validation accuracy)

✿ Model başlangıçta underfitting gösteriyor ama zamanla doğrulama doğruluğu artıyor.
✿ Epoch ilerledikçe doğrulama doğruluğu hızla yükseliyor ve eğitim doğruluğunu geçiyor. Bu, modelin veriyi öğrenmeye başladığını gösteriyor.
✿ Genel olarak overfitting belirtileri fazla yok;

# Eğitim ve Doğrulama Kaybı Grafiği

- `Mavi çizgi:` Eğitim kaybı (train loss)
- `Turuncu çizgi:` Doğrulama kaybı (val loss)

✿ Başlangıçta kayıp yüksek, ilerledikçe düşüyor.
✿ Eğitim kaybı doğrulama kaybının hemen altına düşmediği için model tamamen ezberlememiş, yani regularization (Dropout) işe yaramış.

In [ ]:
import matplotlib.pyplot as plt

# Accuracy grafiği
plt.figure(figsize=(8,5))
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Eğitim ve Doğrulama Doğruluğu')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

# Loss grafiği
plt.figure(figsize=(8,5))
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Eğitim ve Doğrulama Kaybı')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()